In [1]:
import sys
sys.path.append('../')
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from src.models.WolfPQ import WolfPQ
from tqdm.notebook import trange, tqdm


In [2]:
def bvecs_read(fname):
    a = np.fromfile(fname, dtype=np.int32, count=1)
    b = np.fromfile(fname, dtype=np.uint8)
    d = a[0]
    return b.reshape(-1, d + 4)[:, 4:].copy()


def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()


def fvecs_read(fname):
    return ivecs_read(fname).view('float32')

In [17]:
training_data = torch.from_numpy(fvecs_read("/home/catalinlup/MyWorkspace/MasterThesis/ThesisCodebase/big_data/siftsmall/siftsmall_learn.fvecs"))

In [25]:

max_value = training_data.max()
min_value = training_data.min()

def normalize(data):
    return 2 * (data - min_value) / (max_value - min_value) - 1


training_data_norm = normalize(training_data)
print(training_data_norm)

tensor([[ 0.0052, -0.8135, -0.9067,  ..., -0.9793, -0.9585, -0.7306],
        [-0.3057, -0.7720, -0.6788,  ..., -0.9275, -0.9689, -0.9482],
        [-0.6995, -0.9378, -0.6062,  ..., -0.8238, -0.9793, -1.0000],
        ...,
        [-0.9689, -0.8549, -0.7617,  ..., -0.9585, -0.8342,  0.0674],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -0.9896],
        [-1.0000, -0.9896, -0.4093,  ..., -0.9896, -1.0000, -0.9689]])


In [50]:
NUM_EPOCHS = 500
BATCH_SIZE = 500
M = 8
K = 256
DIM = training_data_norm.shape[1]
N = training_data_norm.shape[0]
model = WolfPQ(dim = DIM, M =  M, K = K)
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

In [51]:
def train_one_epoch(epoch_index):
    epoch_loss = 0

    num_batches = training_data_norm.shape[0] // BATCH_SIZE

    for bi in trange(num_batches):

        optim.zero_grad()
        
        batch_data = training_data_norm[bi * BATCH_SIZE: (bi + 1) * BATCH_SIZE, :]
        output, _ = model(batch_data)

        loss = loss_fn(output, batch_data)
        loss.backward()
        optim.step()


        epoch_loss += loss.item()

    epoch_loss /= num_batches

    return epoch_loss
        

In [52]:
model.train(True)

for i in range(NUM_EPOCHS):
    epoch_loss = train_one_epoch(i)
    print(f'Epoch {i}, Loss {epoch_loss}')

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 0, Loss 0.8976583480834961


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1, Loss 0.6980575859546662


  0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch 497, Loss 0.045235396921634675


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 498, Loss 0.0452295358479023


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 499, Loss 0.045172335281968116


In [53]:
model.codebook

Parameter containing:
tensor([[[-1.7527e-01, -7.8225e-01, -9.6807e-01,  ..., -1.5384e-02,
          -2.3590e-01, -6.2500e-01],
         [ 1.7068e-01,  9.2671e-04,  1.4552e-01,  ...,  5.8325e-01,
           4.9908e-01, -5.4766e-01],
         [ 5.6137e-01, -3.2193e-01,  1.6356e+00,  ...,  1.5010e-01,
          -1.6794e-01,  1.1423e+00],
         ...,
         [-1.4054e+00, -2.7513e-01,  2.6350e-01,  ..., -1.7214e+00,
          -1.2162e+00,  1.7276e+00],
         [ 6.5572e-01, -1.8179e+00, -4.4803e-01,  ...,  6.4134e-01,
          -1.3710e-01, -9.2265e-01],
         [-2.7842e-01,  5.8439e-01, -6.4088e-01,  ...,  1.4577e-01,
           1.2890e+00, -8.1435e-01]],

        [[ 7.3318e-01, -1.0440e+00, -2.0491e+00,  ...,  4.0571e-01,
           3.3924e-01,  4.7538e-01],
         [ 3.7002e-01,  2.2551e-01,  5.5136e-01,  ...,  2.9935e-01,
          -5.1028e-01, -1.4120e-02],
         [-5.4862e-01,  4.1685e-01,  2.3147e-01,  ..., -5.2646e-01,
          -2.3809e-01, -2.0482e+00],
         ...,
   

In [54]:
def recall_at_r(I, gt, r):
    """
    Compute Recall@r over the all queries.

    Args:
        I (np.ndarray): Retrieval result, with shape(#queries, ANY), integer.
                        The index of the database item
        gt (np.ndarray): Groundtruth. np.array with shape(#queries, ANY). Integer.
                         Only gt[:, 0] is used
        r (int): Top-r

    Returns:
        The average recall@r over all queries
    """
    assert r <= I.shape[1]
    assert len(I) == len(gt)
    n_ok = (I[:, :r] == gt[:, :1]).sum()
    return n_ok / float(I.shape[0])

def search_query(query: np.ndarray, k: int, vectors:np.ndarray):
    distances = np.sum((vectors - query) ** 2, axis=1)
    sorted_indices = np.argsort(distances)

    top_k_indices = sorted_indices[:k]
    top_k_distances = distances[top_k_indices]

    return top_k_distances, top_k_indices

def search(queries: np.ndarray, k: int, vectors:np.ndarray):
    distance_results = []
    index_results = []

    for i in range(queries.shape[0]):
        query = queries[i]
        d, i = search_query(query, k, vectors)
        distance_results.append(d)
        index_results.append(i)

    return np.vstack(distance_results), np.vstack(index_results)

In [55]:
base = fvecs_read("/home/catalinlup/MyWorkspace/MasterThesis/ThesisCodebase/big_data/siftsmall/siftsmall_base.fvecs")
queries = fvecs_read("/home/catalinlup/MyWorkspace/MasterThesis/ThesisCodebase/big_data/siftsmall/siftsmall_query.fvecs")
ground_truth = ivecs_read("/home/catalinlup/MyWorkspace/MasterThesis/ThesisCodebase/big_data/siftsmall/siftsmall_groundtruth.ivecs")

In [56]:
base_torch = torch.from_numpy(base)
base_torch_normalized = normalize(base_torch)
with torch.no_grad():
    base_quantized, base_indices = model(base_torch_normalized)

In [57]:
base_quantized_np = base_quantized.numpy()
print(torch.argmax(base_indices, dim=-1))

tensor([[165, 225, 124,  ..., 247, 253,  98],
        [218,  49, 151,  ...,  15, 255, 187],
        [230,  49, 124,  ..., 247, 230,  22],
        ...,
        [145, 131, 150,  ..., 117, 230,  67],
        [126, 169, 177,  ..., 251,  67,  72],
        [165, 109,  41,  ...,   1, 167, 249]])


In [58]:
queries_torch = torch.from_numpy(queries)
queries_normalized = normalize(queries_torch)
queries_normalized_np = queries_normalized.numpy()
print(queries_normalized_np)

[[-0.9896373  -0.96891195 -0.88601035 ... -0.5647669  -0.50259066
  -0.88601035]
 [-0.58549225 -0.74093264 -0.88601035 ... -0.96891195 -0.8031088
  -0.865285  ]
 [-0.7098446  -0.9585492  -0.96891195 ... -0.97927463 -0.4404145
  -0.5129534 ]
 ...
 [-0.8549223  -0.91709846 -1.         ... -0.6683938  -0.8549223
  -0.84455955]
 [-0.88601035 -0.50259066 -0.89637303 ... -1.         -1.
  -1.        ]
 [-0.9585492  -0.89637303 -0.9481865  ... -0.55440414 -1.
  -0.91709846]]


In [59]:
distances, I = search(queries_normalized_np, 10, base_quantized_np)

In [60]:
recall_at_r(I, ground_truth, 10)

0.54

In [61]:
torch.save(model.state_dict(), 'model.pt')